In [10]:
import os
import numpy as np
import pandas as pd

In [11]:
df = pd.read_pickle('resources/tdot_testing_incidents.pk')

In [12]:
df.loc[520]

,BLM,County,Route,Co Seq,Location,Year Of Crash,Date of Crash,Time of Crash,Type of Crash,Total Killed,...,Light Conditions,GPS Coordinate Latitude,GPS Coordinate Longitude,Total Other Injuries,Total Veh,ID NUMBER,unit_segment_id,cell_id,time,timestamp
520,0.098,DAVIDSON,I0024,1,Along Roadway,2019,12/19/2019 12:00:00 AM,1006,Suspected Minor Injury,0,...,Daylight,36.33891,-86.91617,1,2,19I0024001,75.0,7.0,2019-12-19 10:06:00,1.576772e+09
520,16.650,DAVIDSON,I0040,1,Along Roadway,2020,3/6/2020 12:00:00 AM,1304,Prop Damage (over),0,...,Daylight,36.16772,-86.79611,0,2,19I0040001,860.0,20.0,2020-03-06 13:04:00,1.583521e+09


In [13]:
# Create an Auxiliary Accident DataFame indicating [Date, Time, Road Segment ID, Number of Accidents]

In [15]:
df_freq = df.groupby(['unit_segment_id', 'Date of Crash', 'Time of Crash', 'GPS Coordinate Latitude', 'GPS Coordinate Longitude', 'timestamp']).size()
df_freq = df_freq.reset_index()
df_freq.columns = ['segment_id', 'date', 'time', 'lat', 'lng', 'timestamp', 'count']

In [16]:
df_freq.date = pd.to_datetime(df_freq.date, infer_datetime_format=True)

In [17]:
min(df_freq.date), max(df_freq.date)

(Timestamp('2019-08-01 00:00:00'), Timestamp('2020-05-01 00:00:00'))

In [18]:
# Extract data for december

In [19]:
df_time_s = df_freq.time.apply(lambda x: '{}:{}:00'.format(str(x)[:-2] if len(str(x)) > 2 else 0, str(x)[-2:]))

df_freq.time = df_freq.date + pd.to_timedelta(df_time_s, unit='h')

df_freq = df_freq.drop('date', axis=1)

In [20]:
df_freq_dec = df_freq[df_freq.time.dt.month == 12]

In [21]:
df_freq_dec.loc[:, 'segment_id'] = df_freq_dec.segment_id.apply(int)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [22]:
df_freq_dec.head()

,segment_id,time,lat,lng,timestamp,count
8,75,2019-12-19 10:06:00,36.33891,-86.91617,1.576772e+09,1
9,75,2019-12-08 14:18:00,36.33207,-86.90093,1.575836e+09,1
10,75,2019-12-08 16:45:00,36.33152,-86.90128,1.575845e+09,1
11,75,2019-12-09 07:04:00,36.33891,-86.91617,1.575897e+09,1
34,78,2019-12-15 14:01:00,36.32500,-86.87300,1.576440e+09,1


In [23]:
df_freq_dec.to_pickle('output/tdot_12_incidents.pkl')